# Evaluation: CrashRepair

This notebook is used to automatically parse the summary of results file for each experiment (`report.json`) and to produce a set of tables and graphs that are used in the paper.

In [ ]:
import json
import os
import typing as t

DIR_RESULTS = os.path.abspath("../results")


def locate_results_files() -> t.List[str]:
    """Returns a list with the absolute paths of each results file."""
    output: t.List[str] = []
    for root, __, files in os.walk(DIR_RESULTS):
        for filename in files:
            if filename == "report.json":
                filename = os.path.join(root, filename)
                output.append(filename)
    return output

def load_result(filename: str) -> t.Dict[str, t.Any]:
    """Loads the results from a given file."""
    assert os.path.isabs(filename)
    result: t.Dict[str, t.Any]
    with open(filename, "r") as fh:
        result = json.load(fh)
    result["scenario"] = os.path.basename(os.path.dirname(filename))
    result["program"] = os.path.basename(os.path.dirname(os.path.dirname(filename)))
    return result

def load_results(*, _results=[]) -> t.List[t.Dict[str, t.Any]]:
    if _results:
        return _results
    for filename in locate_results_files():
        _results.append(load_result(filename))
    return _results

### RQ3: Repair Operators

The table below shows the relative effectiveness of each our repair operators in terms of the number of plausible and correct patches it generates.

In [ ]:
# Operator | # Candidates | # Plausible Candidates (%) | # Correct Candidates (%) | # Scenarios Plausibly Fixed | # Scenarios Correctly Fixed
from dataclasses import dataclass, field


@dataclass
class OperatorSummary:
    name: str
    num_candidates: int = 0
    num_plausible_candidates: int = 0
    plausible_scenarios: t.Set[str] = field(default_factory=set)
    # CT: note that, for now, we have no automatic means of assessing patch correctness
    # correct_candidates: int = 0
    # correct_scenarios: int = 0

    @property
    def num_plausible_scenarios(self) -> int:
        return len(self.plausible_scenarios)

    @property
    def percent_plausible_candidates(self) -> float:
        return (self.num_plausible_candidates / self.num_candidates) * 100


def compute_operator_summaries() -> t.List[OperatorSummary]:
    results = load_results()
    operator_to_summary: t.Dict[str, OperatorSummary] = {}

    for scenario_results in results:
        if "validation" not in scenario_results:
            continue
        if "generation" not in scenario_results:
            continue
        if "evaluations" not in scenario_results["validation"]:
            continue
        if "candidates" not in scenario_results["generation"]:
            continue

        scenario_name = scenario_results["scenario"]
        candidates = scenario_results["generation"]["candidates"]
        evaluations = scenario_results["validation"]["evaluations"]

        for evaluation in evaluations:
            candidate = candidates[evaluation["patch-id"]]
            operator = candidate["operator"]

            if operator not in operator_to_summary:
                operator_to_summary[operator] = OperatorSummary(operator)
            summary = operator_to_summary[operator]

            summary.num_candidates += 1
            if evaluation["is-repair"]:
                summary.num_plausible_candidates += 1
                summary.plausible_scenarios.add(scenario_name)

    return list(operator_to_summary.values())

def compute_operator_table() -> None:
    summaries = compute_operator_summaries()
    table: t.List[t.List[str]] = [
        ["Operator", "# Candidates", "# Plausible Candidates", "% Plausible", "# Plausible Scenarios"],
    ]
    for summary in summaries:
        row = [
            summary.name,
            summary.num_candidates,
            summary.num_plausible_candidates,
            f"{summary.percent_plausible_candidates:.2f}%",
            summary.num_plausible_scenarios,
        ]
        table.append(row)

    print(table)

compute_operator_table()